In [1]:
import findspark

In [2]:
import re

In [3]:
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')

In [4]:
import pyspark

In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *

In [6]:
sc = pyspark.SparkContext()

In [7]:
spark = SparkSession \
    .builder \
    .appName("PA3") \
    .config("spark.some.config.option","some-value") \
    .getOrCreate()

In [27]:
import pandas as pd
df = pd.read_csv("train.csv")
df_test =  pd.read_csv("test.csv")

In [28]:
df = spark.createDataFrame(df)
df_test = spark.createDataFrame(df_test)

In [29]:
df1 =df.withColumn('plot',lower(regexp_replace(df['plot'],"[^a-zA-Z ]","")))
df1_test =df_test.withColumn('plot',lower(regexp_replace(df_test['plot'],"[^a-zA-Z ]","")))

In [30]:
from nltk.corpus import stopwords
import nltk 
nltk.download('stopwords')
df1.take(1)

[nltk_data] Downloading package stopwords to /home/cse587/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[Row(movie_id=23890098, movie_name='Taxi Blues', plot='shlykov a hardworking taxi driver and lyosha a saxophonist develop a bizarre lovehate relationship and despite their prejudices realize they arent so different after all', genre="['World cinema', 'Drama']")]

In [31]:
stop = stopwords.words('english')
df1.printSchema()


root
 |-- movie_id: long (nullable = true)
 |-- movie_name: string (nullable = true)
 |-- plot: string (nullable = true)
 |-- genre: string (nullable = true)



In [32]:
df1 = df1.dropna()
df1_test = df1_test.dropna()

In [33]:
df2 = df1.select('plot')
df2_test = df1_test.select('plot')

In [54]:
rd = df2.rdd.flatMap(list)
rd_test = df2_test.rdd.flatMap(list)
data = df2.rdd.map(list)
data_test = df2_test.rdd.map(list)
data = data.map(lambda x: x[0].split())
data_test = data_test.map(lambda x: x[0].split())
def dd(x):
    to_remove = []
    for word in x:
        if word not in stop:
            continue
        else:
            to_remove.append(word)
    for word in to_remove:
        x.remove(word)
    return x
data = data.map(dd)
data_test = data_test.map(dd)
data.take(1)


[['shlykov',
  'hardworking',
  'taxi',
  'driver',
  'lyosha',
  'saxophonist',
  'develop',
  'bizarre',
  'lovehate',
  'relationship',
  'despite',
  'prejudices',
  'realize',
  'arent',
  'different']]

In [36]:
rd1 = rd.flatMap(lambda x: x.split())
rd1_test = rd_test.flatMap(lambda x:x.split())
rd1 = rd1.filter(lambda x: x not in stop)
rd1_test = rd1_test.filter(lambda x: x not in stop)

In [37]:
m = rd1.map(lambda x: (x,1))
m_test = rd1_test.map(lambda x: (x,1))

In [38]:
wordCount = m.reduceByKey(lambda a,b: a+b)
wordCount_test = m_test.reduceByKey(lambda a,b: a+b)

In [39]:
common_words = wordCount.filter(lambda x: x[1] > 200)
common_words_test = wordCount_test.filter(lambda x: x[1] > 200)

In [40]:
common_words.count()

4312

In [41]:
features = common_words.keys()
features_test = common_words_test.keys()

In [42]:
from pyspark .mllib.linalg import SparseVector

In [43]:
from collections import Counter
def remove_uncommon_words(x):
    for word in x:
        if word in features:
            continue
        else:
            x.remove(word)
    return x


In [44]:
features= features.collect()
features_test= features_test.collect()

In [57]:
data = data.map(remove_uncommon_words)
data_test = data_test.map(remove_uncommon_words)

In [46]:
vocab_map = data.flatMap(lambda token: token).distinct() \
    .zipWithIndex().collectAsMap()

vocab_map_test = data_test.flatMap(lambda token: token).distinct() \
    .zipWithIndex().collectAsMap()

In [67]:
v_map = sc.broadcast(vocab_map)
vocab_size = sc.broadcast(len(vocab_map))
tdm = data \
    .map(Counter) \
    .map(lambda counts: {v_map.value[token]: float(counts[token]) for token in counts})\
    .map(lambda index_counts: SparseVector(vocab_size.value, index_counts))


v_map_test = sc.broadcast(vocab_map_test)
vocab_size_test = sc.broadcast(len(vocab_map_test))
tdm_test = data \
    .map(Counter) \
    .map(lambda counts: {v_map_test.value[token]: float(counts[token]) for token in counts})\
    .map(lambda index_counts: SparseVector(vocab_size_test.value, index_counts))

In [69]:
labels = df1.select('genre')

In [70]:
labels= labels.rdd.flatMap(lambda x:x)

In [71]:
labels.take(3)

["['World cinema', 'Drama']",
 "['Action/Adventure', 'Action', 'Science Fiction', 'Drama']",
 "['Musical', 'Action', 'Drama']"]

In [ ]:
sparseVectorsList = tdm.collect()
sparseVectorsList_Test = tdm_test.collect()

In [ ]:
data.take(1)

In [ ]:
movie_ids = df.select('movie_id')
movie_ids = movie_ids.rdd
movie_ids = movie_ids.flatMap(lambda x:x)
movie_ids = movie_ids.collect()


In [ ]:
keylist = pd.read_csv('mapping.csv')
keylist.rename(columns={'Unnamed: 0': 'i'}, inplace=True)
keylist = spark.createDataFrame(keylist)
k = keylist.select('0')
k1 = keylist.select('i')
k = k.rdd.flatMap(lambda x:x)
k1 = k1.rdd.flatMap(lambda x:x)
genre_list = k1.zip(k)

In [ ]:
genre_list.collect()

In [ ]:
import numpy as np
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import SVMWithSGD, SVMModel


In [ ]:
# def mapFunc(line):
#     val = 0.0
#     vals = line.map(lambda x: (x[0], x[1]))
#     for lab in vals[0]:
#         if label == lab:
#             val = 1.0
#             break
#     return LabeledPoint(val, vals[1])

def test(data):
    ans = []
    for i in range(len(data)):
        val = 0.0
        labels = data[i][0].replace('[','').replace(']','').replace('\'','').split(',')
        for lab in labels:
            if lab == label:
                val = 1.0
                break
        pt = LabeledPoint(val, data[i][1])
        ans.append(pt)
    return ans

def makeClassifier(data):
#     lab2Data = data.map(mapFunc)
    lst = test(data)
    return SVMWithSGD.train(sc.parallelize(lst), iterations=100)
#     l2dl = lab2Data.collect()
#     return SVMWithSGD.train(sc.parallelize(l2dl), iterations=100)

In [ ]:
lab2sparse = labels.zip(tdm)
lab2sparse.take(1)

In [ ]:
l2sm = lab2sparse.map(lambda x: (x[0],x[1]))
data = l2sm.collect()

In [ ]:
data[0][0]

In [ ]:
classifiers = []

for label in k.collect():
    print('Trained', label, 'Classifier')
    classifiers.append(makeClassifier(data))
# classifier = makeClassifier(lab2sparse)

# Part 2: TF-IDF

In [ ]:
 from pyspark.mllib.feature import HashingTF, IDF